In [12]:
# an implementation of value iteration to solve for the optimal policy
# for the recycling robot that is described on page 52 of the RL book.

S = [1,2]    # state is lo, hi
A = [1,2,3]  # action is search, wait, recharge

# transition probabilities when searching
# when action = search
#       lo   hi
#    -------------
# lo |  β   1-β
# hi |1-α    α
α = 0.2
β = 0.3
r_search = 5.0   # expected reward when searching
r_wait = 1.0

# the 4-argument transition probabilities p(sprime,r|s,a).
# let's try to get by with the 3-argument probabilities p(sprime|s,a)
# that are shown in the table on page 52.
# the indexing is p[s, a, sprime]
p3 = zeros(2, 3, 2)
p3[2,1,2] = α
p3[2,1,1] = 1-α
p3[1,1,2] = 1-β
p3[1,1,1] = β
p3[2,2,2] = 1
p3[1,2,1] = 1
p3[1,3,2] = 1
# now note that when the state=hi, action=recharge will not be considered.
# we will take care of that in the value iteration function.

# an implementation of the value iteration algorithm shown on page
# 83 in the RL book.
function value_iteration(V, π)
    γ = 0.9    # discount factor
    θ = 1e-6   # tolerance for convergence
    k = 0      # iteration/sweep
    while true
        δ = 0.0
        for s in 1:length(S)
            v = V[s]  # old value
            q = zeros(length(A))  # State-action values
            for a in 1:length(A)
                if s == 2 && a == 3 #skip actions such as recharging in a high state
                    continue
                end
                reward = (a == 1 ? r_search : r_wait) #reward for actions
                for s_prime in 1:length(S)
                    q[a] += p3[s, a, s_prime] * (reward + γ * V[s_prime]) #computation of q given rewards and s'
                end
            end
            V[s], best_action = findmax(q) #update value function
            π[s, setdiff(A, best_action)] .= 0.0
            π[s, best_action] = 1.0 #update policy
            δ = max(δ, abs(v - V[s])) #max change
        end
        k += 1  # Increment iteration counter
        if δ < θ
            break
        end
    end
    println(k, " sweeps.")
    return V, π
end


# the policy: rows correspond to states, columns correspond to actions.
π = fill(1.0 / length(A), length(S), length(A))
V = zeros(length(S))  # will hold the value function
V, π = value_iteration(V, π)



97 sweeps.


([49.999995093705294, 49.99999551296493], [1.0 0.0 0.0; 1.0 0.0 0.0])